<h1> LTestChiSquaredTests.ipynb </h1>

This code is prepared so that it condenses all the information from the code files inside TestAModelFolder/ML. 

First copy and paste the folders that CrystallineMLAlone.ipynb and AmorphousMLAlone.ipynb have created inside LTestVisualCheck/Models_to_be_predicted.
Then run all the cells (skip the last one if you don't want images that take a lot of time and space) if you want to obtain information using the chi squared metric. These cells will create a subfolder called Results where only three files are important:
1. **Chi2\_rank\_vertical.png** For every experiment, the chi squared value of the preditions of each model is obtained. Then all chi squared values of the different models in each experiment is compared. The one that has the smaller chi squared value obtains a +1, the second smallest a +2 and so on. This process is repeated for each experiment and these integer values are summed. At the end we can say that the model with the smallest score has performed better overall
2. **Chi2\_AccumulativeScores.txt** has the same information than the graph but on a txt file
3. **Chi2\_NumberOfExperimentsWhereEachModelOutperforms.txt** as its name indicates counts only the number of experiment where each model obtains the smallest chi squared value. Sometimes, models that overfit perform better on this test (as they pass through all the points)


If you run the last one it will automatically combine the predictions for each experiment in a single image. This way it is easier to spot the differences between models in a quick look.

<H1> CHI SQUARED TEST </H1>

In [ ]:
import os
import pandas as pd
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import numpy as np
import math
from pathlib import Path
from PIL import Image
import re
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def long_path(path):
    """
    Arguments:
        path (path): The path that needs to be converted

    Returns:
        The updated path string or path depending on the platform used

    Notes:
        To avoid Windows 260 character limit for Windows paths, a special "prefix" is added.
        It also unifies how directories are managed.
        Also works with Linux and Mac

    """
    # Convert to Path and resolve to absolute
    path = Path(path).resolve()

    #Windows only:
    if os.name == "nt":
        path_str = str(path)
        if not path_str.startswith("\\\\?\\"):
            if path_str.startswith("\\\\"):
                path_str = "\\\\?\\UNC\\" + path_str[2:]
            else:
                path_str = "\\\\?\\" + path_str
            return path_str

    return path
def compute_chi2_for_experiment(df_real, df_pred):
    """
    Compute the reduced chi-squared value as the sum of tipyfied normal distributions over the number of distributions
    
        χ^2 = (1/N) * \sum [ (pred - real)^2 / (err_real)^2 ]
    """
    if len(df_real) != len(df_pred):
        raise ValueError("Real and predicted files must have the same length.")
    
    N = len(df_real)
    diff = df_pred["PredictedPolarizationD3"].values - df_real["RealPolarizationD3"].values
    err = df_real["ErrRealPolarizationD3"].values
    err[err == 0] = 1e-10  # avoid division by zero
    
    chi2 = np.sum((diff**2) / (err**2)) / N
    return chi2


def load_data(experiment_folder):
    """
    Load the txt files containing real and predicted data
    for a MissingPolarizationD3_* experiment.
    """
    experiment_folder = long_path(experiment_folder)
    files = [long_path(os.path.join(experiment_folder, f)) for f in os.listdir(experiment_folder)]

    real_file = next(f for f in files if os.path.basename(f).startswith("RawData_PolarizationD3_MissingPolarizationD3_"))
    pred_file = next(f for f in files if os.path.basename(f).startswith("PredictedPoints_PolarizationD3_MissingPolarizationD3_"))

    df_real = pd.read_csv(real_file, sep="\t")
    df_pred = pd.read_csv(pred_file, sep="\t")
    return df_real, df_pred


def count_missing_folders(model_path):
    """
    Return the list of MissingPolarizationD3_* subfolders for a given model folder.
    """
    model_path = long_path(model_path)
    return sorted([
        f for f in os.listdir(model_path)
        if f.startswith("MissingPolarizationD3_") and os.path.isdir(long_path(os.path.join(model_path, f)))
    ])


def save_chi2_values_to_txt(model_folder_name, missing_folders, chi2_values, output_dir):
    """
    Save the chi-squared values to a structured text file
    inside the 'Results' folder at the root level.
    """
    chi2_dir = long_path(os.path.join(output_dir, "Results"))
    os.makedirs(chi2_dir, exist_ok=True)  # ensure folder exists

    filename = f"Chi2_values_{model_folder_name}.txt"
    filepath = long_path(os.path.join(chi2_dir, filename))

    with open(filepath, "w") as f:
        for folder_name, chi2 in zip(missing_folders, chi2_values):
            f.write(f"{folder_name}: {chi2:.6f}\n")

    print(f"File saved: {filepath}")


def main_all_models(root_folder):
    root_folder = long_path(root_folder)

    models_root = long_path(os.path.join(root_folder, "Models_to_be_compared"))

    if not os.path.isdir(models_root):
        print(f"Models folder not found: {models_root}")
        return

    all_model_folders = [
        d for d in os.listdir(models_root)
        if os.path.isdir(long_path(os.path.join(models_root, d)))
    ]

    if not all_model_folders:
        print("No model folders found.")
        return

    for model_folder in all_model_folders:
        model_path = long_path(os.path.join(models_root, model_folder))
        missing_folders = count_missing_folders(model_path)

        if not missing_folders:
            print(f" No MissingPolarizationD3_ folders in {model_folder}")
            continue

        chi2_values = []
        for missing_folder in missing_folders:
            experiment_path = long_path(os.path.join(model_path, missing_folder))
            try:
                df_real, df_pred = load_data(experiment_path)
                chi2 = compute_chi2_for_experiment(df_real, df_pred)
                chi2_values.append(chi2)
            except Exception as e:
                print(f"   Error in {missing_folder}: {e}")

        if chi2_values:
            save_chi2_values_to_txt(model_folder, missing_folders, chi2_values, root_folder)
            chi2_mean = np.mean(chi2_values)
        else:
            print(f"No valid chisquared values computed for {model_folder}.")

main_all_models(r".")
folder_path = long_path("Results")
if not os.path.isdir(folder_path):
    print(f"Folder not found: {folder_path}")
    print("   Tip: run the computation cell first so it creates Results/ and the files.")
def is_result_file(name: str) -> bool:
    return (
        name.endswith(".txt") and
        (name.startswith("Chi2_values_") or name.startswith("ChiSquared_values_"))
    )

# Initialize dictionaries
Chi2Values = {}             
ExperimentName = OrderedDict() 

# Collect files
result_files = []
if os.path.isdir(folder_path):
    for filename in os.listdir(folder_path):
        filename = long_path(os.path.join(folder_path, filename))  # wrap early
        if is_result_file(os.path.basename(filename)):
            result_files.append(filename)

result_files.sort()
if not result_files:
    print("No results found in ChiSquared/ matching 'Chi2_values_*.txt' or 'ChiSquared_values_*.txt'.")
else:
    for full_path in result_files:
        with open(full_path, "r") as f:
            lines = f.readlines()

        Chi2Values[os.path.basename(full_path)] = {}
        for line in lines:
            if ":" in line:
                try:
                    experiment_name, value = line.split(":", 1)
                    experiment_name = experiment_name.strip()
                    value = float(value.strip())
                    Chi2Values[os.path.basename(full_path)][experiment_name] = value
                except ValueError:
                    print(f"Skipping malformed line in {full_path}: {line.strip()}")

        if not ExperimentName and Chi2Values[os.path.basename(full_path)]:
            for i, experiment_name in enumerate(Chi2Values[os.path.basename(full_path)].keys(), 1):
                ExperimentName[experiment_name] = f"exp{i}"

plt.figure(figsize=(14, 7))
exp_ids = list(ExperimentName.values())
assert len(Chi2Values) >= 2, "You need at least two chisquared values files"
files = list(Chi2Values.keys())

try:
    cmap = plt.colormaps.get_cmap('tab20')  # new API
except AttributeError:
    cmap = plt.get_cmap('tab20')  # old API

colors = cmap.colors
line_styles = ['-', '--', ':', '-.', (0, (5, 1)), (0, (3, 1, 1, 1))]
line_widths = [2] * len(files)

for j, filename in enumerate(files):
    chi2_values = [
        Chi2Values[filename].get(exp_name, np.nan)  # Use NaN if missing
        for exp_name in ExperimentName.keys()
    ]
    
    plt.plot(
        range(len(exp_ids)),
        chi2_values,
        color=colors[j % len(colors)],
        linestyle=line_styles[j % len(line_styles)],
        linewidth=line_widths[j],
        label=filename
    )

xticks_pos = [i for i in range(len(exp_ids)) if (i + 1) % 10 == 0]
xticks_labels = [exp_ids[i] for i in xticks_pos]
plt.xticks(xticks_pos, xticks_labels, rotation=45, ha='right')

plt.xlabel("Experiment", fontsize=12)
plt.ylabel(r"$\chi^2$", fontsize=12)
plt.title(r"Chi-squared values per experiment (model comparison)", fontsize=14)
plt.grid(alpha=0.2, linestyle='--')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

def normalize_key(k):
    return k.replace("-", "_")

Chi2Values_norm = {
    model: {normalize_key(exp): val for exp, val in Chi2Values[model].items()}
    for model in Chi2Values
}
ExperimentName_norm = {normalize_key(exp): exp for exp in ExperimentName.keys()}
all_chi2_values = np.array([
    [Chi2Values_norm[f].get(exp, np.nan) for exp in ExperimentName_norm.keys()]
    for f in Chi2Values_norm.keys()
])


exp_ids = [ExperimentName_norm[exp] for exp in ExperimentName_norm.keys()]
model_ids = list(Chi2Values_norm.keys())  
files = list(Chi2Values.keys())  
nb_models = len(files)
exp_ids = list(ExperimentName.keys())  

all_chi2_values = []
for f in files:
    model_vals = []
    for exp in exp_ids:
        if exp not in Chi2Values[f]:
            print(f"Warning: experiment '{exp}' missing for model '{f}'")
            model_vals.append(np.nan) 
        else:
            model_vals.append(Chi2Values[f][exp])
    all_chi2_values.append(model_vals)
all_chi2_values = np.array(all_chi2_values)


avg_chi2 = np.nanmean(all_chi2_values, axis=0)  
try:
    cmap = plt.colormaps['tab20']   
except (AttributeError, TypeError):
    cmap = plt.get_cmap('tab20')  

colors = cmap(range(nb_models))

# Plot deviation from the average
plt.figure(figsize=(18, 12))
for j in range(nb_models):
    diff_from_avg = all_chi2_values[j] - avg_chi2
    plt.plot(
        range(1, len(exp_ids) + 1), 
        diff_from_avg,
        linewidth=2,
        color=colors[j % len(colors)],
        label=files[j]
    )

xticks_pos = [k + 1 for k in range(len(exp_ids)) if (k + 1) % 10 == 0]
plt.xticks(xticks_pos, rotation=0)
plt.xlabel("Experiment Index", fontsize=12)
plt.ylabel("ΔChisuared (model - average)", fontsize=12)
plt.title("Deviation of Chi squared values from experiment-wise average", fontsize=14)
plt.grid(alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


# Save number of wins (lowest Chi squared per experiment) 
output_folder = long_path("Results")
os.makedirs(output_folder, exist_ok=True)

model_scores = {f: 0 for f in files}
for chi2_vals in all_chi2_values.T:
    if np.all(np.isnan(chi2_vals)):
        continue
    min_index = np.nanargmin(chi2_vals)
    winning_model = files[min_index]
    model_scores[winning_model] += 1

sorted_scores = sorted(model_scores.items(), key=lambda x: x[1], reverse=True)

output_file = long_path(os.path.join(output_folder, "Chi2_NumberOfExperimentsWhereEachModelOutperforms.txt"))
with open(output_file, "w") as f:
    f.write("Approval Rate (Number of wins by minimal chisquared. The higher the better):\n\n")
    for model, score in sorted_scores:
        f.write(f"{model}: {score} points\n")
print(f"Results saved to {output_file}")


# Save rank-based accumulative scores
all_chi2_values = []
for f in files:
    model_chi2 = []
    for exp in exp_ids:
        model_chi2.append(Chi2Values[f].get(exp, np.nan))
    all_chi2_values.append(model_chi2)
all_chi2_values = np.array(all_chi2_values)

model_scores = {f: 0 for f in files}
for chi_values in all_chi2_values.T:
    sorted_indices = np.argsort(chi_values)  # best to worst
    for rank, model_idx in enumerate(sorted_indices, start=1):
        model_name = files[model_idx]
        model_scores[model_name] += rank

sorted_scores = sorted(model_scores.items(), key=lambda x: x[1])

output_file = long_path(os.path.join(output_folder, "Chi2_AccumulativeScores.txt"))
with open(output_file, "w") as f:
    f.write("Approval Rate (Score by rank. Lower is better):\n\n")
    for model, score in sorted_scores:
        f.write(f"{model}: {score} points\n")
print(f"Scores saved to {output_file}")

output_folder = long_path("Results")
os.makedirs(output_folder, exist_ok=True)
model_scores = {f: 0.0 for f in files}
for chi2_vals in all_chi2_values.T:
    if np.all(np.isnan(chi2_vals)):
        continue
    min_chi2 = np.nanmin(chi2_vals)
    for idx, val in enumerate(chi2_vals):
        if np.isnan(val):
            score = 0.0
        else:
            score = min_chi2 / val 
        model_name = files[idx]
        model_scores[model_name] += score

# Normalize by number of experiments
for model in model_scores:
    model_scores[model] /= len(exp_ids)

# Sort models (higher = better)
sorted_scores = sorted(model_scores.items(), key=lambda x: x[1], reverse=True)

# --- Save results ---
output_file = long_path(os.path.join(output_folder, "Chi2_RelativeNormalizedScores.txt"))
with open(output_file, "w") as f:
    f.write("Relative Score (Normalized chisqaured per experiment. Higher is better):\n\n")
    for model, score in sorted_scores:
        f.write(f"{model}: {score:.4f}\n")
print(f"Normalized relative scores saved to {output_file}")


# Vertical bar chart: Rank-based scoring (lower = better)
sorted_scores_rank = sorted(model_scores.items(), key=lambda x: x[1])
models = [m for m, _ in sorted_scores_rank]
scores = [s for _, s in sorted_scores_rank]

norm = plt.Normalize(min(scores), max(scores))
try:
    cmap = plt.colormaps['RdYlGn_r']
except (AttributeError, TypeError):
    cmap = plt.get_cmap('RdYlGn_r')

colors = cmap(norm(scores))
short_models = [m.replace("Chi2_values_AllTestsFolder_", "").replace(".txt", "") for m in models]
short_models = ["_".join(m.split("_")[-2:]) for m in short_models]

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(short_models, scores, color=colors)

for bar, score in zip(bars, scores):
    ax.text(
        bar.get_x() + bar.get_width()/2,
        score + score * 0.02,
        f"{score:.3f}".replace(',', '.'),
        ha='center', va='bottom',
        fontsize=8
    )

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(scores)
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Rank Score (Lower = Better)")

ax.set_ylabel("Total Rank Score", fontsize=12)
ax.set_xlabel("Model", fontsize=12)
ax.set_title("Approval Rate: Rank-Based Scoring (Chi², Green=Better, Red=Worse)", fontsize=14)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(long_path(os.path.join(output_folder, "Chi2_rank_vertical.png")), dpi=600, bbox_inches='tight')
plt.show()


# Vertical bar chart: Relative normalized scoring (higher = better)
sorted_scores_rel = sorted(model_scores.items(), key=lambda x: x[1], reverse=True)
models = [m for m, _ in sorted_scores_rel]
scores = [s for _, s in sorted_scores_rel]

short_models = [m.replace("Chi2_values_AllTestsFolder_", "").replace(".txt", "") for m in models]
short_models = ["_".join(m.split("_")[-2:]) for m in short_models]

norm = plt.Normalize(min(scores), max(scores))
try:
    cmap = plt.colormaps['RdYlGn']
except (AttributeError, TypeError):
    cmap = plt.get_cmap('RdYlGn')

colors = cmap(norm(scores))

fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(short_models, scores, color=colors, edgecolor='black')

for bar, score in zip(bars, scores):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        score + score * 0.02,
        f"{score:.3f}".replace(',', '.'),
        ha='center', va='bottom',
        fontsize=8
    )

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(scores)
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Relative Score (Higher = Better)")

ax.set_ylabel("Total Relative Score (averaged over experiments)", fontsize=12)
ax.set_xlabel("Model", fontsize=12)
ax.set_title("Approval Rate: Normalized Relative Scoring (chisquared, Green = Best, Red = Worst)", fontsize=14)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
ax.set_ylim(0, max(scores) * 1.1)

plt.tight_layout()
plt.savefig(long_path(os.path.join(output_folder, "Chi2_relative_vertical.png")), dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# Pair models with scores
model_scores = list(zip(short_models, scores))

# Sort by score (ascending)
model_scores_sorted = sorted(model_scores, key=lambda x: x[1])
print("Models sorted by score (lowest -> highest):")
i=0
for model, score in model_scores_sorted:
    i=i+1
    print(f"{model}: {score:.3f}")
print(i)
sorted_models, sorted_scores = zip(*model_scores_sorted)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

model_scores = list(zip(short_models, scores))

# Sort by score (ascending -> worst at top, best at bottom)
model_scores_sorted = sorted(model_scores, key=lambda x: x[1])

# Unzip
sorted_models, sorted_scores = zip(*model_scores_sorted)

# Define colormap (red → yellow → green)
cmap = cm.get_cmap("RdYlGn")
norm = mcolors.Normalize(vmin=min(sorted_scores), vmax=max(sorted_scores))
colors = [cmap(norm(s)) for s in sorted_scores]
fig, ax = plt.subplots(figsize=(10, len(sorted_models) * 0.15))
bars = ax.barh(sorted_models, sorted_scores, color=colors, edgecolor="black")

ax.set_xlabel("Total Relative Score", fontsize=12)
ax.set_ylabel("Model", fontsize=12)
ax.set_title("Approval Rate: Sorted Scores (Red = Worst, Green = Best)", fontsize=14)
ax.grid(axis="x", alpha=0.3)

ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=7)

for bar, score in zip(bars, sorted_scores):
    ax.text(
        bar.get_width() + max(sorted_scores) * 0.01,
        bar.get_y() + bar.get_height() / 2,
        f"{score:.3f}".replace(',', '.'),
        va="center", ha="left", fontsize=8
    )

# Remove top/bottom empty space
ax.set_ylim(-0.5, len(sorted_models) - 0)

# Extend X-axis so labels fit
ax.set_xlim(0, max(sorted_scores) * 1.1)
"""
# Add colorbar legend
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Relative Score (Higher = Better)")
"""
plt.tight_layout()
results_folder = long_path("Results")
os.makedirs(results_folder, exist_ok=True)
plt.savefig(
    long_path(os.path.join(results_folder, "FullVerticalChiCrystalline.png")),
    dpi=900,
    bbox_inches="tight"
)

plt.show()




In [ ]:
def extract_number(model_name):
    match = re.search(r'_(\d+)$', model_name)
    return int(match.group(1)) if match else float('inf')

model_scores = list(zip(short_models, scores))

# Sort by numeric suffix (ascending: 1 -> N)
model_scores_sorted = sorted(model_scores, key=lambda x: extract_number(x[0]))

sorted_models, sorted_scores = zip(*model_scores_sorted)

cmap = cm.get_cmap("RdYlGn")
norm = mcolors.Normalize(vmin=min(sorted_scores), vmax=max(sorted_scores))
colors = [cmap(norm(s)) for s in sorted_scores]

fig, ax = plt.subplots(figsize=(10, len(sorted_models) * 0.5))
bars = ax.barh(sorted_models, sorted_scores, color=colors, edgecolor="black")
ax.set_xlabel("Total Relative Score", fontsize=12)
ax.set_ylabel("Model", fontsize=12)
ax.set_title("Approval Rate: Ordered by Model Number (Red = Worst, Green = Best)", fontsize=14)
ax.grid(axis="x", alpha=0.3)

ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=7)

for bar, score in zip(bars, sorted_scores):
    ax.text(
        bar.get_width() + max(sorted_scores) * 0.01,
        bar.get_y() + bar.get_height() / 2,
        f"{score:.3f}".replace(',', '.'),
        va="center", ha="left", fontsize=8
    )

ax.set_ylim(-0.5, len(sorted_models) - 0.5)
ax.set_xlim(0, max(sorted_scores) * 1.1)

plt.tight_layout()
results_folder = long_path("Results")
os.makedirs(results_folder, exist_ok=True)
plt.savefig(
    long_path(os.path.join(results_folder, "FullHorizontalChiCrystalline.png")),
    dpi=900,
    bbox_inches="tight"
)

plt.show()


<h1> PLOTS COMPARING THE MODELS</h1>

In [ ]:
root_dir = Path.cwd()
models_root = root_dir / "Models_to_be_compared"
output_dir = root_dir / "Results" / "ExperimentComparison"
output_dir.mkdir(parents=True, exist_ok=True)

# Helpers
def best_subplot_shape(n):
    """Balanced grid for n images."""
    cols = math.ceil(math.sqrt(n))
    rows = math.ceil(n / cols)
    return rows, cols

def extract_model_name(folder_name: str) -> str:
    """
    Extract model name from something like:
    CrystallineAllTestsFolder_Naif81016_3
    """
    match = re.search(r"Naif[^_]+", folder_name)
    return match.group(0) if match else folder_name

def shorten_experiment_name(exp_folder: str) -> str:
    """
    Shorten experiment folder names for filenames.
    """
    return exp_folder.replace("Missing", "").strip("_")

# Find all Missing*.jpg images
all_images = list(models_root.rglob("Missing*.jpg"))
print(f"Found {len(all_images)} images total.")

# Group images by experiment
experiments = {}

for img_path in all_images:
    exp_name = img_path.parent.name         
    model_folder = img_path.parents[1].name 

    model_name = extract_model_name(model_folder)

    experiments.setdefault(exp_name, []).append(
        (model_name, img_path)
    )

print(f"Found {len(experiments)} experiments.")

# Build comparison figures
for exp_name, model_images in experiments.items():

    model_images.sort(key=lambda x: x[0])
    n_models = len(model_images)
    if n_models == 0:
        continue
    rows, cols = best_subplot_shape(n_models)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    axes = axes.flatten() if n_models > 1 else [axes]

    for ax, (model_name, img_path) in zip(axes, model_images):
        lp = long_path(img_path)
        img = Image.open(lp)
        ax.imshow(img)
        ax.set_title(model_name, fontsize=8)
        ax.axis("off")

    for ax in axes[n_models:]:
        ax.axis("off")

    plt.tight_layout()

    save_name = shorten_experiment_name(exp_name) + ".png"
    save_path = output_dir / save_name
    plt.savefig(save_path, dpi=600, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved {save_path}")
